<a href="https://colab.research.google.com/github/srehaag/crs-summer-2024/blob/main/CRS_Summer_Course_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CRS Summer Course -- GenAI

This notebook aims to help students in the 2024 version of the CRS Summer Course understand how generative AI can be used as a research tool.

#### Setup

In [1]:
!pip install -q datasets
!pip install -q langchain
!pip install -q langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0

#### Authenticate with OpenAI

Instructions:

1.   Get OpenAI developper platform account: https://platform.openai.com/signup
2.   Create & copy API key: https://platform.openai.com/settings/profile?tab=api-keys
3.   If using Colab, click on the key icon on the left, select add new secret, name the secret OPENAI_API_KEY & paste the key into the value field

In [2]:
# Authenticate with OpenAI using API key (for Colab users)
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

#### Load Refugee Law Lab Reporter Dataset from Huggingface

For further details, see: https://refugeelab.ca/bulk-data

In [3]:
# Load Data
from datasets import load_dataset
df = load_dataset("refugee-law-lab/canadian-legal-data", "RLLR", split="train").to_pandas()
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/539 [00:00<?, ? examples/s]

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
0,2022 RLLR 32,VC1-05483,RLLR,2022,,en,2022/03/08,https://refugeelab.ca/rllr/2022rllr32,2024-04-11,Citation: 2022 RLLR 32\nTribunal: Refugee Prot...,"{""country"": ""Mexico"", ""case_type"": ""PSG: Gende..."
1,2022 RLLR 31,VC1-06798,RLLR,2022,,en,2022/02/03,https://refugeelab.ca/rllr/2022rllr31,2024-04-11,Citation: 2022 RLLR 31\nTribunal: Refugee Prot...,"{""country"": ""Sudan"", ""case_type"": ""Political O..."
2,2022 RLLR 30,VC2-07212,RLLR,2022,,en,2022/11/28,https://refugeelab.ca/rllr/2022rllr30,2024-04-11,Citation: 2022 RLLR 30\nTribunal: Refugee Prot...,"{""country"": ""Barbados"", ""case_type"": ""PSG: Gen..."
3,2022 RLLR 29,VC2-06405,RLLR,2022,,en,2022/11/09,https://refugeelab.ca/rllr/2022rllr29,2024-04-11,Citation: 2022 RLLR 29\nTribunal: Refugee Prot...,"{""country"": ""Ukraine"", ""case_type"": ""PSG: Gend..."
4,2022 RLLR 28,VC2-06395,RLLR,2022,,en,2022/09/28,https://refugeelab.ca/rllr/2022rllr28,2024-04-11,Citation: 2022 RLLR 28\nTribunal: Refugee Prot...,"{""country"": ""India"", ""case_type"": ""Political O..."


In [4]:
# See example of text of a case
unofficial_text = df['unofficial_text'][250]
print (unofficial_text)

Citation: 2020 RLLR 74
Tribunal: Refugee Protection Division
Date of Decision: July 7, 2020
Panel: Avril Cardoso
Counsel for the Claimant(s): Atul Subedi
Country: Nepal
RPD Number: TB8-19093
Associated RPD Number(s): TB8-19105, TB8-19141, TB8-19142
ATIP Number: A-2021-00800
ATIP Pages: 000092-000099
REASONS FOR DECISION
[1] [XXX] (the principal claimant), [XXX] (associate claimant), [XXX] (adult claimant) and [XXX] (minor claimant) claim to be a citizen of Nepal and are claiming refugee protection pursuant to section 96 and subsection 97(1) of the Immigration and Refugee Protection Act (“the Act”)[1].
ALLEGATIONS
[2] The claimants’ allegations are fully set out in their Basis of Claim forms (BOC)[2]. In summary, the principal claimant alleges a fear of persecution in Nepal because of his political membership and active support of the Nepali Congress (NC) party. The associate, adult and minor claimants allege a fear of persecution because of their imputed political opinion as members of

#### Set up OpenAI calls via API

In [5]:
# Function to send true/false questions about a case to OpenAI GPT4o
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=1)
system_prompt = "You are a helpful assistant for a Canadian refugee law professor. You are provided with a case and asked a question about the case. You only answer true or false."

def get_response(system_prompt, question, unofficial_text):
  human_prompt = f"""CASE: {unofficial_text}\nQUESTION: {question}\nANSWER: """
  messages = [("system", system_prompt),("human", human_prompt)]
  return llm.invoke(messages).content

### Use OpenAI GPT4o to answer true/false questions about legal texts

In [6]:
is_trans = "Is this a case about a refugee claim that primarily involves alleged persecution on account of the claimaint being transgender?"
get_response(system_prompt, is_trans, unofficial_text)

'False'

In [7]:
# get 20 random case and see if is trans
df = df.sample(n=20, random_state=888).reset_index(drop=True)
df["is_trans"] = df["unofficial_text"].apply(lambda x: get_response(system_prompt, is_trans, x))
df.head()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other,is_trans
0,2020 RLLR 167,MB9-23336,RLLR,2020,,en,2020/02/21,https://refugeelab.ca/rllr/2020rllr167,2024-04-11,Citation: 2020 RLLR 167\nTribunal: Refugee Pro...,"{""country"": ""Saudi Arabia"", ""case_type"": ""PSG:...",True
1,2019 RLLR 87,TB7-18122,RLLR,2019,,en,2019/01/10,https://refugeelab.ca/rllr/2019rllr87,2024-04-11,Citation: 2019 RLLR 87\n\nTribunal: Refugee Pr...,"{""country"": ""Libya"", ""case_type"": ""Political O...",False
2,2019 RLLR 109,TB8-28669,RLLR,2019,,en,2019/11/26,https://refugeelab.ca/rllr/2019rllr109,2024-04-11,Citation: 2019 RLLR 109\n\nTribunal: Refugee P...,"{""country"": ""Eritrea"", ""case_type"": ""Political...",False
3,2020 RLLR 7,MB7-24231,RLLR,2020,,en,2020/01/28,https://refugeelab.ca/rllr/2020rllr7,2024-04-11,Citation: 2020 RLLR 7\nTribunal: Refugee Prote...,"{""country"": ""Burundi"", ""case_type"": ""Political...",False
4,2020 RLLR 159,TB9-19705,RLLR,2020,,en,2020/02/04,https://refugeelab.ca/rllr/2020rllr159,2024-04-11,Citation: 2020 RLLR 159\nTribunal: Refugee Pro...,"{""country"": ""India"", ""case_type"": ""PSG: SOGIE""...",False


In [8]:
# Show how many cases are trans cases
df.is_trans.value_counts()

is_trans
False    19
True      1
Name: count, dtype: int64

In [9]:
# Print the cases where is trans is true
trans_cases = df[df["is_trans"] == "True"]["unofficial_text"]
for case in trans_cases:
    print(case)

Citation: 2020 RLLR 167
Tribunal: Refugee Protection Division
Date of Decision: February 21, 2020
Panel: François Ramsay
Counsel for the Claimant(s): Jessica Lipes
Country: Saudi Arabia
RPD Number: MB9-23336
Associated RPD Number(s): N/A
ATIP Number: A-2022-00978
ATIP Pages: 000064-000068
REASONS FOR DECISION
INTRODUCTION
[1] These are the reasons for the decision in the claim of XXXX XXXX XXXX XXXX, (also known as XXXX XXXX), who claims to be a citizen of Saudi Arabia, and is claiming refugee protection pursuant to sections 96 and 97(1) of the Immigration and Refugee Protection Act.
[2] This claim has been decided without a hearing, according to the Immigration and Refugee Board’s Chairperson’s Instructions Governing the Streaming of Less Complex Claims at the Refugee Protection Division (RPD) and paragraph 170(f) of the Act.
[3] In rendering my reasons, I have considered and applied the Chairperson’s Guidelines on Proceedings Before the IRB Involving Sexual Orientation and Gender Ide

In [10]:
is_granted = "Was the applicant successful in obtaining refugee protection in the case? Answer only true or false."
get_response(system_prompt, is_granted, unofficial_text)

'True'

In [11]:
df["is_granted"] = df["unofficial_text"].apply(lambda x: get_response(system_prompt, is_granted, x))
df.is_granted.value_counts()

is_granted
True    20
Name: count, dtype: int64